In [ ]:
from PIL import Image
import os
from pytesseract import pytesseract
import math
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

: 

In [30]:
files = os.listdir("/Users/gaurav/Desktop/Coding/Split-Bill")

In [31]:
#Open image with PIL
img = Image.open(find_image_path(files))

#Extract text from image
text = pytesseract.image_to_string(img)

In [32]:
def find_image_path(files):
    for file in files:
        if file[-4:]=="jpeg":
            image_path = file
            
    return image_path

In [33]:
#split the text into a list by line break
def split_by_line(text):
    content = []
    ctr = 0
    prev_pos = 0
    for i in range(len(text)):
        if text[i] == '\n':
            content.append(text[prev_pos+1:i])
            ctr = ctr + 1
            prev_pos = i
    content = list(filter(None, content))
    return content

In [34]:
text = split_by_line(text)

In [35]:
#Find Index that contains a string
def find_index_of_string(text, string):
    index = [i for i, s in enumerate(text) if string in s][0]
    return index

In [36]:
#get price from line on reciept
def get_price(money):
    money = float(money.split('$')[1])
    return money

In [37]:
index_sub_total = find_index_of_string(text,'Subtotal')
sub_total = get_price(text[index_sub_total])

In [ ]:
#find index of list where the items begin
def find_start_of_items(text, index, sub_total):
    total = []
    for num in reversed(range(index)):
        total.append(float(text[num].split('$')[1]))
        if "{:.2f}".format((math.fsum(total))) == "{:.2f}".format(sub_total):
            start = num
            break
    
    return start

: 

In [39]:
start = find_start_of_items(text, index_sub_total, sub_total)

# make main dishes into pandas dataframe
def main_dishes_pandas (text, start,index_sub_total):
    items = text[start:index_sub_total]
    number_of_items = []
    dish = []
    cost = []
    
    for item in items:
        number_of_items.append(item[0])
        name = item.split('$')[0]
        dish.append(name[1:])
        cost.append(float(item.split('$')[1]))
        
    df = pd.DataFrame({'num':number_of_items,'dish': dish,'cost':cost})
    return df

In [40]:
df =  main_dishes_pandas(text, start,index_sub_total)

In [41]:
def who_ordered_what(df):
    names= []
    for index, row in df.iterrows():
        for num in row.num:
            names.append(input('Who ordered' + row.dish))
    df['name'] = names
    
    return df

In [45]:
df = who_ordered_what(df)
df

Who ordered %Chicken  adam
Who ordered ¥%Chicken  gaurav
Who ordered Chicken Caesar Wrap  adam
Who ordered %Chicken  Kevin


,num,dish,cost,name
0,1,%Chicken,20.32,adam
1,1,¥%Chicken,18.30,gaurav
2,1,Chicken Caesar Wrap,14.45,adam
3,1,%Chicken,22.92,Kevin


In [46]:
def handling_duplicate_names(df):
    df['total'] = df.groupby(['name'])['cost'].transform('sum')
    df = df.drop_duplicates(subset='name')
    df = df[['name','total']]
    return df

In [47]:
df = handling_duplicate_names(df)

In [54]:
def get_tax_total(text, sub_total):  
    Total_index = find_index_of_string(text,'Total')
    total = get_price(text[Total_index])
    tax = total - sub_total
    return tax

In [60]:
def format_receipt(df,tax): 
    taxes = list()
    for index, row in df.iterrows():
        percentage = (row.total/sum(df.total))
        taxes.append(percentage*tax)
    df["taxes"] = taxes
    df["amount to be paid"] = (df["taxes"] + df["total"])
    df = df.pivot_table(index='name',
                   margins=True,
                   margins_name='total',  # defaults to 'All'
                   aggfunc=sum)
    
    return df

In [61]:
tax = get_tax_total(text,sub_total)
df = format_receipt(df,tax)
df

,amount to be paid,taxes,total
name,,,
Kevin,19.36,-3.56,22.92
adam,29.37,-5.40,34.77
gaurav,15.46,-2.84,18.30
total,64.19,-11.80,75.99
